In [1]:
import pandas as pd
import numpy as np
from nameparser import HumanName
import re
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
dfroster = pd.read_csv('YaleW_group.csv')
dflinkedin = pd.read_csv('CornellWomenLinkedIn.csv')

In [3]:
dfroster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 17 columns):
FirstLast      272 non-null object
Criteria       272 non-null object
FirstLast.1    272 non-null object
Year           272 non-null int64
YearsPlayed    272 non-null int64
RosterLink     56 non-null object
Hometown       167 non-null object
HS             167 non-null object
ClassYear      167 non-null object
College        272 non-null object
First          272 non-null object
Gender         272 non-null object
Last           272 non-null object
MidInit        3 non-null object
MidName        3 non-null object
Name           167 non-null object
RosterYear     167 non-null object
dtypes: int64(2), object(15)
memory usage: 36.2+ KB


In [4]:
dfroster['Gender'] = dfroster['Gender'] == 'M'
dfroster['Gender'] = dfroster['Gender'].astype(int)

In [5]:
dflinkedin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 8 columns):
Name            201 non-null object
LName           201 non-null object
SearchResult    201 non-null object
Headline        201 non-null object
Location        201 non-null object
Company         195 non-null object
School          201 non-null object
Activites       201 non-null object
dtypes: object(8)
memory usage: 12.7+ KB


In [6]:
dflinkedin.head()

,Name,LName,SearchResult,Headline,Location,Company,School,Activites
0,['Megan Polnet'],Megan Polnet,https://www.linkedin.com/in/megan-polnet-27224...,Student Athlete,"North Miami Beach, Florida",NaN,['Cornell University - College of Arts and Sci...,[]
1,['Sarah Campbell'],Sarah Campbell,https://www.linkedin.com/in/sarah-campbell-241...,Student Athlete,Greater Boston Area,Nawshawtuc Country Club,"['Cornell University', 'Lincoln-Sudbury Region...","[""Women's Varsity Tennis (2018-present), Big R..."
2,['Michelle Deng'],Michele Marass,https://www.linkedin.com/in/michele-marass/,PhD | Scientific Editor at Nature Communicatio...,"London, Greater London, United Kingdom",Springer Nature,"['Max Planck Society', 'Scuola Internazionale ...",['Member of the International Max Planck Resea...
3,['Ashley Huang'],Rob Reid,https://www.linkedin.com/in/rwreid/,Research Assistant Professor at University of ...,"Harrisburg, North Carolina",Not Found,[],[]
4,['Valerie Ho'],Valerie Ho,https://www.linkedin.com/in/valeriehho/,Student-Athlete at Cornell University,Washington D.C. Metro Area,Sodexo,"['Cornell University', 'Montgomery Blair High ...","[""Varsity Women's Tennis, Big Red Leadership I..."


In [7]:
# Remove the brackets and single-quotes from the 'Name' string in each cell of the column 
dflinkedin['Name'] = dflinkedin['Name'].str.strip('[]').str.strip("''")


In [8]:
#dflinkedin['Name']

In [9]:
# Manually correct bad spelling from roster download
dflinkedin['Name'][12] = 'Maria Perez de la Llave'
dflinkedin['Name'][36] = 'Lusiné Mehrabyan'
dflinkedin['Name'][85] = 'Kim Gantz'


In [10]:
# First remove the brackets from the 'Name' string, then remove single quotes
# Optionally remove double space from Middle Initial/Name calc 
#dflinkedin['Name'] = dflinkedin['Name'].str.strip('[]').astype(str)
#dflinkedin['Name'] = dflinkedin['Name'].str.strip("''").astype(str)
#dflinkedin['Name'] = dflinkedin['Name'].str.replace('  ', ' ')

In [11]:
dflinkedin.head()

,Name,LName,SearchResult,Headline,Location,Company,School,Activites
0,Megan Polnet,Megan Polnet,https://www.linkedin.com/in/megan-polnet-27224...,Student Athlete,"North Miami Beach, Florida",NaN,['Cornell University - College of Arts and Sci...,[]
1,Sarah Campbell,Sarah Campbell,https://www.linkedin.com/in/sarah-campbell-241...,Student Athlete,Greater Boston Area,Nawshawtuc Country Club,"['Cornell University', 'Lincoln-Sudbury Region...","[""Women's Varsity Tennis (2018-present), Big R..."
2,Michelle Deng,Michele Marass,https://www.linkedin.com/in/michele-marass/,PhD | Scientific Editor at Nature Communicatio...,"London, Greater London, United Kingdom",Springer Nature,"['Max Planck Society', 'Scuola Internazionale ...",['Member of the International Max Planck Resea...
3,Ashley Huang,Rob Reid,https://www.linkedin.com/in/rwreid/,Research Assistant Professor at University of ...,"Harrisburg, North Carolina",Not Found,[],[]
4,Valerie Ho,Valerie Ho,https://www.linkedin.com/in/valeriehho/,Student-Athlete at Cornell University,Washington D.C. Metro Area,Sodexo,"['Cornell University', 'Montgomery Blair High ...","[""Varsity Women's Tennis, Big Red Leadership I..."


In [12]:
dflinkedin['fuzz_ratio'] = dflinkedin.apply(lambda x: fuzz.ratio(x['Name'], x['LName']), axis=1)
dflinkedin['fuzz_pratio'] = dflinkedin.apply(lambda x: fuzz.partial_ratio(x['Name'], x['LName']), axis=1)
dflinkedin['fuzz_tokensort'] = dflinkedin.apply(lambda x: fuzz.token_sort_ratio(x['Name'], x['LName']), axis=1)
dflinkedin['fuzz_tokenset'] = dflinkedin.apply(lambda x: fuzz.token_set_ratio(x['Name'], x['LName']), axis=1)
dflinkedin['fuzz_wratio'] = dflinkedin.apply(lambda x: fuzz.WRatio(x['Name'], x['LName']), axis=1)


In [13]:
# Hardcode College value "Cornell" but later merge roster wtih LinkedIn and use field from Roster
dflinkedin['tennis_yn'] = dflinkedin['Activites'].str.lower().str.contains("tennis")
dflinkedin['School_yn'] = dflinkedin['School'].str.contains("Cornell")
dflinkedin['SameName_yn'] = (dflinkedin['Name'] == dflinkedin['LName'])
dflinkedin['SameName_yn'][dflinkedin['Name'].isna()] = np.nan
dflinkedin.tennis_yn = dflinkedin.tennis_yn.astype(int)
dflinkedin.School_yn = dflinkedin.School_yn.astype(int)
dflinkedin.SameName_yn = dflinkedin.SameName_yn.astype(int)
dflinkedin['SameTennis_yn'] = dflinkedin['tennis_yn'] + dflinkedin['SameName_yn'] 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
dflinkedin.rename(columns={'Name':'FirstLast'}, inplace=True)

In [15]:
dflinkedin.describe()

,fuzz_ratio,fuzz_pratio,fuzz_tokensort,fuzz_tokenset,fuzz_wratio,tennis_yn,School_yn,SameName_yn,SameTennis_yn
count,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000
mean,77.751244,82.238806,77.870647,83.218905,82.935323,0.273632,0.676617,0.477612,0.751244
std,27.291717,25.541783,27.317818,26.900778,24.052251,0.446935,0.468935,0.500746,0.746867
min,8.000000,10.000000,7.000000,7.000000,19.000000,0.000000,0.000000,0.000000,0.000000
25%,64.000000,64.000000,61.000000,71.000000,74.000000,0.000000,0.000000,0.000000,0.000000
50%,88.000000,100.000000,90.000000,100.000000,95.000000,0.000000,1.000000,0.000000,1.000000
75%,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000,1.000000,1.000000,1.000000
max,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000,1.000000,1.000000,2.000000


In [16]:
dflinkedin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 17 columns):
FirstLast         201 non-null object
LName             201 non-null object
SearchResult      201 non-null object
Headline          201 non-null object
Location          201 non-null object
Company           195 non-null object
School            201 non-null object
Activites         201 non-null object
fuzz_ratio        201 non-null int64
fuzz_pratio       201 non-null int64
fuzz_tokensort    201 non-null int64
fuzz_tokenset     201 non-null int64
fuzz_wratio       201 non-null int64
tennis_yn         201 non-null int64
School_yn         201 non-null int64
SameName_yn       201 non-null int64
SameTennis_yn     201 non-null int64
dtypes: int64(9), object(8)
memory usage: 26.8+ KB


In [17]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [18]:
df_suffix = pd.merge(dfroster, dflinkedin,
                     left_on='FirstLast', right_on='FirstLast', how='outer',suffixes=('_R','_L'))

In [19]:
df_suffix['Decade'] = df_suffix['Year']//10*10

In [20]:
df_suffix.drop(columns=['FirstLast.1'], inplace=True)

In [21]:
df_suffix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 0 to 271
Data columns (total 33 columns):
FirstLast         272 non-null object
Criteria          272 non-null object
Year              272 non-null int64
YearsPlayed       272 non-null int64
RosterLink        56 non-null object
Hometown          167 non-null object
HS                167 non-null object
ClassYear         167 non-null object
College           272 non-null object
First             272 non-null object
Gender            272 non-null int64
Last              272 non-null object
MidInit           3 non-null object
MidName           3 non-null object
Name              167 non-null object
RosterYear        167 non-null object
LName             201 non-null object
SearchResult      201 non-null object
Headline          201 non-null object
Location          201 non-null object
Company           195 non-null object
School            201 non-null object
Activites         201 non-null object
fuzz_ratio        201 non-nul

In [22]:
df_suffix.to_csv('RosterLinkedIn_CornellW.csv', index=False)

In [23]:
df_suffix

,FirstLast,Criteria,Year,YearsPlayed,RosterLink,Hometown,HS,ClassYear,College,First,Gender,Last,MidInit,MidName,Name,RosterYear,LName,SearchResult,Headline,Location,Company,School,Activites,fuzz_ratio,fuzz_pratio,fuzz_tokensort,fuzz_tokenset,fuzz_wratio,tennis_yn,School_yn,SameName_yn,SameTennis_yn,Decade
0,Katherine Nguyen,Cornell,2020,2,https://cornellbigred.com/sports/womens-tennis...,"Fountain Valley, Calif.",Capistrano Connections Academy,['Freshman' 'Sophomore'],Cornell University,Katherine,0,Nguyen,NaN,NaN,Katherine Nguyen,"[""2018-19 Women's Tennis Roster"" ""2019-20 Wome...",Katherine Nguyen,https://www.linkedin.com/in/katherine-nguyen-b...,Student-Athlete at Cornell University,Greater Los Angeles Area,SmartyPants Vitamins,"['Cornell University', 'Fountain Valley High S...","[""Cornell Varsity Women's Tennis Team, Big Red...",100.0,100.0,100.0,100.0,100.0,1.0,1.0,1.0,2.0,2020
1,Ashley Huang,Cornell,2020,1,https://cornellbigred.com/sports/womens-tennis...,"Wexford, Pa.",North Allegheny HS,['Freshman'],Cornell University,Ashley,0,Huang,NaN,NaN,Ashley Huang,"[""2019-20 Women's Tennis Roster""]",Rob Reid,https://www.linkedin.com/in/rwreid/,Research Assistant Professor at University of ...,"Harrisburg, North Carolina",Not Found,[],[],10.0,12.0,20.0,20.0,21.0,0.0,0.0,0.0,0.0,2020
2,Valerie Ho,Cornell,2020,2,https://cornellbigred.com/sports/womens-tennis...,"Silver Spring, Md.",Montgomery Blair HS,['Freshman' 'Sophomore'],Cornell University,Valerie,0,Ho,NaN,NaN,Valerie Ho,"[""2018-19 Women's Tennis Roster"" ""2019-20 Wome...",Valerie Ho,https://www.linkedin.com/in/valeriehho/,Student-Athlete at Cornell University,Washington D.C. Metro Area,Sodexo,"['Cornell University', 'Montgomery Blair High ...","[""Varsity Women's Tennis, Big Red Leadership I...",100.0,100.0,100.0,100.0,100.0,1.0,1.0,1.0,2.0,2020
3,Khary Pryce,Cornell,2020,2,https://cornellbigred.com/sports/womens-tennis...,"Ellicott City, Md.",McDonogh School,['Freshman' 'Sophomore'],Cornell University,Khary,0,Pryce,NaN,NaN,Khary Pryce,"[""2018-19 Women's Tennis Roster"" ""2019-20 Wome...",Khary Pryce,https://www.linkedin.com/in/khary-pryce-543470...,Peer Facilitator at Intergroup Dialogue Proje...,"Albany, New York Area",Cornell University,"['Cornell University', 'IMG Academy', 'McDonog...","['Varsity Tennis Team, Big Red Leadership Inst...",100.0,100.0,100.0,100.0,100.0,1.0,1.0,1.0,2.0,2020
4,Fatima El Ashram,Cornell,2020,1,https://cornellbigred.com/sports/womens-tennis...,"Zamalek, Cairo, Egypt",Weil College Prep,['Freshman'],Cornell University,Fatima,0,Ashram,El,El,Fatima El Ashram,"[""2019-20 Women's Tennis Roster""]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
5,Maria Adiaconitei,Cornell,2020,2,https://cornellbigred.com/sports/womens-tennis...,"Bethesda, Md.",Laurel Springs HS,['Freshman' 'Sophomore'],Cornell University,Maria,0,Adiaconitei,NaN,NaN,Maria Adiaconitei,"[""2018-19 Women's Tennis Roster"" ""2019-20 Wome...",Maria Adiaconitei,https://www.linkedin.com/in/maria-adiaconitei-...,Student at Cornell University,"Ithaca, New York Area",Washington Orthopaedics and Sports Medicine,['Cornell University'],"['Varsity Tennis Team, Big Red Leadership Inst...",100.0,100.0,100.0,100.0,100.0,1.0,1.0,1.0,2.0,2020
6,Megan Polnet,Cornell,2020,1,https://cornellbigred.com/sports/womens-tennis...,"Churchville, Pa.",Laurel Springs,['Freshman'],Cornell University,Megan,0,Polnet,NaN,NaN,Megan Polnet,"[""2019-20 Women's Tennis Roster""]",Megan Polnet,https://www.linkedin.com/in/megan-polnet-27224...,Student Athlete,"North Miami Beach, Florida",NaN,['Cornell University - College of Arts and Sci...,[],100.0,100.0,100.0,100.0,100.0,0.0,1.0,1.0,1.0,2020
7,Michelle Deng,Cornell,2020,1,https://cornellbigred.com/sports/womens-tennis...,"Temple City, Calif.",Arcadia HS,['Freshman'],Cornell University,Michelle,0,Deng,NaN,NaN,Michelle Deng,"[""2019-20 Women's Tennis Roster""]",Michele Marass,https://www.linkedin.com/in/michele-marass/,PhD | Scientific Editor at Natur